In [ ]:
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import gensim
import os, re, csv, math, codecs
num_words=30000
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=num_words+2,)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

In [ ]:
num_words=len(np.unique(np.hstack(data)))
print("Categories:", np.unique(targets))
print("Number of unique words:", num_words)

In [ ]:
length = [len(i) for i in data]
print("Average Review length:", np.mean(length))
print("Standard Deviation:", round(np.std(length)))

In [ ]:
print("Label:", targets[0])
Label: 1
print(data[0])

In [ ]:
index = imdb.get_word_index()
reverse_index = dict([(value, key) for (key, value) in index.items()])
decoded = " ".join( [reverse_index.get(i - 3, "#") for i in data[1]] )
print(decoded)

In [ ]:
#load embeddings
EMBEDDING_DIR = "./"
print('loading word embeddings...')
embeddings_index = {}
f = codecs.open(EMBEDDING_DIR+'glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_index))

In [ ]:
embeddings_index["car"]

In [ ]:
embed_dim=100
embedding_matrix=np.zeros([num_words+4,embed_dim])
for word, idx in index.items():
  if idx <= num_words and word in embeddings_index:
    embedding_matrix[idx+3,:]=embeddings_index[word]

In [ ]:
maxlen=1000

In [ ]:
data = pad_sequences(data,maxlen=maxlen)


In [ ]:
len(data[0])

In [ ]:
len(data[1])

In [ ]:
data=np.array(data)

In [ ]:
data.shape


In [ ]:
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout, Dense
from keras.models import Sequential
from keras import optimizers
nb_words=num_words+4
num_filters=64
model = Sequential()
model.add(Embedding(nb_words, embed_dim, weights=[embedding_matrix], input_length=maxlen, trainable=False))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters*2, 7, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  #multi-label (k-hot encoding)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()


In [ ]:
model.fit(data,targets,batch_size=32,epochs=20,validation_split=0.2)

In [ ]:
embedding_matrix